# Cleaning Public Support data

## Importing necessary libraries

In [1]:
pip install pandas matplotlib seaborn wordcloud


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime,date, timedelta

## Loading the data

In [2]:
slack = pd.read_csv('../sources/support-channels.csv')


## Discover

In [4]:
print('Shape of slack dataframe before cleaning:', slack.shape)

Shape of slack dataframe before cleaning: (481, 14)


In [5]:
slack.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481 entries, 0 to 480
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Channel_ID        481 non-null    object
 1   Channel_Slug      481 non-null    object
 2   Timestamp         481 non-null    object
 3   Timestamp_Thread  368 non-null    object
 4   User_ID           481 non-null    object
 5   Full_Name         470 non-null    object
 6   Email             481 non-null    object
 7   Permalink         481 non-null    object
 8   Text              481 non-null    object
 9   Text_raw          481 non-null    object
 10  Slack_username    481 non-null    object
 11  Team_ID           481 non-null    object
 12  Team_Name         481 non-null    object
 13  Is_Bot            481 non-null    bool  
dtypes: bool(1), object(13)
memory usage: 49.4+ KB


**Creating 2 new columns**

In [3]:
slack['Is_a_question'] = np.where(slack['Timestamp_Thread'].isnull(), 1, 0)

In [4]:
support_agents = ['1','5301']

slack['Is_agent'] = np.where(slack['User_ID'].isin(support_agents), 1, 0)

**Encoding necessary columns**

In [6]:
slack['Is_Bot'] = np.where(slack['Is_Bot'] == True, 1, 0)

**Converting timestamp columns**

In [7]:
slack['Datetime'] = pd.to_datetime(slack['Timestamp'])
slack['Datetime_Thread'] = pd.to_datetime(slack['Timestamp_Thread'])

**Creating 2 dataframes: questions and answers**

In [8]:
questions_df = slack[slack['Is_a_question'] == 1]
answers_df = slack[slack['Is_a_question'] == 0]

**Working on Answers dataframe**

In [8]:
answers = answers_df.groupby(['User_ID','Datetime'])[['Text']]

In [12]:
df2 = pd.DataFrame(answers.sum().reset_index())

In [13]:
df2.head()

,User_ID,Datetime,Text
0,U01KGAER1TM,2022-11-04 17:02:51,No se quiere usar un tercero para las fotos
1,U01KGAER1TM,2022-11-04 17:04:57,digamos que son imagenes de usuarios en donde ...
2,U01KGAER1TM,2022-11-04 17:05:02,comentarios y likes
3,U01KGAER1TM,2022-11-04 17:06:30,y la base de datos no solo contiene las imagen...
4,U01KGAER1TM,2022-11-04 17:26:59,https://isn365.com/


In [14]:
df2['Diff_in_Seconds'] = (df2.sort_values('Datetime').groupby('User_ID').Datetime.diff())

In [15]:
df2['Diff_in_Seconds'] = df2['Diff_in_Seconds'].fillna(pd.Timedelta(seconds=0))

In [16]:
df2['Diff_in_Seconds'] = df2['Diff_in_Seconds']/np.timedelta64(1,'s')

In [17]:
df2['diff_abs'] = df2.Diff_in_Seconds.abs()

In [18]:
df2['same_author'] = df2['User_ID'].ne(df2['User_ID'].shift().bfill()).astype(int)

In [19]:
def create_AnswerId():
    for group in df2.groupby(['User_ID']):
        df2['messageId'] = df2.diff_abs.gt(300).cumsum() + 1 + df2.same_author.cumsum()

create_AnswerId()

/tmp/ipykernel_2008/749061550.py:2: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for group in df2.groupby(['User_ID']):


In [20]:
df2.shape

(368, 7)

In [21]:
df2.head(2)

,User_ID,Datetime,Text,Diff_in_Seconds,diff_abs,same_author,messageId
0,U01KGAER1TM,2022-11-04 17:02:51,No se quiere usar un tercero para las fotos,0.0,0.0,0,1
1,U01KGAER1TM,2022-11-04 17:04:57,digamos que son imagenes de usuarios en donde ...,126.0,126.0,0,1


**Questions dataframe**

In [10]:
questions = questions_df.groupby(['User_ID','Datetime'])[['Text']]

In [11]:
df3 = pd.DataFrame(questions.sum().reset_index())

In [12]:
df3.shape

(112, 3)

In [25]:
df3['Diff_in_Seconds'] = (df3.sort_values('Datetime').groupby('User_ID').Datetime.diff())

In [25]:
df3['Diff_in_Seconds'] = df3['Diff_in_Seconds'].fillna(pd.Timedelta(seconds=0))

In [26]:
df3['Diff_in_Seconds']=df3['Diff_in_Seconds']/np.timedelta64(1,'s')

In [28]:
df3['diff_abs'] = df3.Diff_in_Seconds.abs()

In [29]:
df3['same_author'] = df3['User_ID'].ne(df3['User_ID'].shift().bfill()).astype(int)

In [30]:
def create_QuestionId():
    for group in df3.groupby(['User_ID']):
        df3['messageId'] = df3['diff_abs'].gt(300).cumsum() + 1 + df3.same_author.cumsum()

create_QuestionId()

/tmp/ipykernel_2008/706305890.py:2: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for group in df3.groupby(['User_ID']):


In [31]:
df3.shape

(112, 7)

In [32]:
df3.tail()

,User_ID,Datetime,Text,Diff_in_Seconds,diff_abs,same_author,messageId
107,U04A6KV066M,2022-11-08 20:08:45,"Hi all,",NaN,NaN,1,89
108,U04A6KV066M,2022-11-08 20:13:24,Hi all. I am trying to split my view to where ...,279.0,279.0,0,89
109,U04A6KV066M,2022-11-08 21:58:27,Should we be doing anything with “commit” and ...,6303.0,6303.0,0,90
110,U6MR8LG4Q,2022-11-05 03:11:30,Hey @betomasia12 No need to make the pictures ...,NaN,NaN,1,91
111,UL08NNSV8,2022-11-01 18:39:22,pareciera que tienes una version de ruby muy a...,NaN,NaN,1,92


In [13]:
df3.head(5)

,User_ID,Datetime,Text
0,U01KGAER1TM,2022-11-04 16:14:32,"Como estan, tengo un problema, ocupo subir una..."
1,U01SJ480RBR,2022-11-07 16:59:39,Hello
2,U01SM5J4MMG,2022-11-07 21:15:26,"Buenas tardes, intenta poniendo solo git push"
3,U02N1P8CV6W,2022-10-31 22:55:36,"Hola chicos, he estado intentando llevar a pro..."
4,U02NE11UHNC,2022-11-03 22:28:10,have someone imported custom fonts (no google ...


In [16]:
answers_df.groupby(['User_ID','Datetime'])[['Text']]

**Merge each dataframe to its previous columns**

In [ ]:
#next steps: 
# 
# 1.verify in questions_df that they match exactly with df3 (as shape has chenged by 1 row)---->randomly verify some questions
# 2.merge df2 oand df3 to some of the columns of its previous df. (https://stackoverflow.com/questions/17978133/python-pandas-merge-only-certain-columns)
# 3. concat text values according to this:
# https://www.statology.org/pandas-combine-rows-with-same-column-value/
# 4. download both datasets and take to Looker to verify they are ok to build graphs

In [37]:
answers_df = answers_df.merge(df2, left_on='Text', right_on='Text')

(368, 7)

In [ ]:
df3.merge(questions_df, left_on='Text', right_on='Text')

**Merge rows that have the same messageId**

**Cleaning joined dataframe**